# **Setup**

In [16]:
from pyspark.sql.functions import col, lit, to_date
from datetime import date

# Parametri
key_date = date(2021, 1, 1)
plant = "PL01"

stko = spark.read.table("STKO")
stpo = spark.read.table("STPO")
mast = spark.read.table("MAST")



StatementMeta(, ad97ab4a-3284-4e83-9cca-c5dee32cdb4d, 18, Finished, Available, Finished)

# **Caricamento Tabelle**

In [17]:
stko = spark.read.table("STKO")
stpo = spark.read.table("STPO")
mast = spark.read.table("MAST")


StatementMeta(, ad97ab4a-3284-4e83-9cca-c5dee32cdb4d, 19, Finished, Available, Finished)

# **Filtro per key_date e plant, join iniziale**

In [18]:
# Join STKO + MAST (per materiale/plant)
bom_header = mast.join(stko, on=["MATNR", "WERKS", "STLNR", "STLAL"], how="inner") \
    .filter((col("WERKS") == plant) & (col("DATUV") <= lit(key_date)))

bom_level1 = bom_header.join(stpo, on="STLNR", how="inner") \
    .filter(col("STPO.DATUV") <= lit(key_date)) \
    .select(
        col("MATNR").alias("ROOT_MAT"),
        col("IDNRK").alias("CHILD_MAT"),
        col("MENGE"),
        lit(1).alias("LEVEL")
    )


StatementMeta(, ad97ab4a-3284-4e83-9cca-c5dee32cdb4d, 20, Finished, Available, Finished)

# **Ricorsione multilivello**

In [19]:
seen = set()
current_level = bom_level1
all_levels = current_level

for level in range(2, 5):  # livello 2 → 4
    filtered = current_level.filter(~col("CHILD_MAT").isin(seen))
    seen.update(filtered.select("CHILD_MAT").rdd.map(lambda r: r[0]).collect())

    next_level = filtered.alias("a").join(
        stpo.alias("b"),
        col("a.CHILD_MAT") == col("b.STLNR"),
        how="inner"
    ).filter(col("b.DATUV") <= lit(key_date)) \
     .select(
        col("a.ROOT_MAT"),
        col("b.IDNRK").alias("CHILD_MAT"),
        col("b.MENGE"),
        lit(level).alias("LEVEL")
     )

    all_levels = all_levels.union(next_level)
    current_level = next_level


StatementMeta(, ad97ab4a-3284-4e83-9cca-c5dee32cdb4d, 21, Finished, Available, Finished)

# **Salvataggio tabella esplosa**

In [20]:
all_levels.write.mode("overwrite").format("delta").saveAsTable("BOM_EXPLODED")


StatementMeta(, ad97ab4a-3284-4e83-9cca-c5dee32cdb4d, 22, Finished, Available, Finished)

# **Output**

In [21]:
display(all_levels.orderBy("ROOT_MAT", "LEVEL"))

StatementMeta(, ad97ab4a-3284-4e83-9cca-c5dee32cdb4d, 23, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8185f9c2-8944-4888-9228-ace47146b6ef)